<a href="https://colab.research.google.com/github/kazeidk/GDrive_Turbo_Copy/blob/main/GDrive_Turbo_Copy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 GDrive Turbo Copy - MAX POWER Edition

## ✨ Tính năng:
- ⚡ **Copy song song** - Lên đến 15 luồng cho A100
- 🎯 **Auto-detect** - Tự nhận diện GPU/RAM & set luồng MAX
- 🔄 **Auto-resume** - Tự tiếp tục khi timeout
- 🔁 **Auto-retry** - Tự retry khi rate limit
- 📊 **Progress + ETA** - Hiển thị tiến độ
- 🔔 **Thông báo** - Âm thanh khi xong

---

## 📖 Hướng dẫn:
1. Chạy cell **Cài đặt**
2. Chạy cell **Nhập thông tin** - auto detect & set MAX
3. Chạy cell **Run**

## 🔥 Recommend setting Colab:
- GPU: **A100** | RAM cao: **BẬT**

In [ ]:
#@title 1️⃣ Cài đặt thư viện
!pip install tqdm -q
print("✅ Đã cài đặt xong!")

In [ ]:
#@title 2️⃣ Nhập thông tin
from ipywidgets import widgets, Layout, HBox
from IPython.display import display, HTML
import psutil

# Auto-detect Colab tier - TURBO MODE
def detect_colab_tier():
    try:
        ram_gb = psutil.virtual_memory().total / (1024**3)
        if ram_gb > 100:  # A100 với High RAM = 167GB
            return 'A100 TURBO', 15, ram_gb
        elif ram_gb > 50:  # A100 thường
            return 'A100', 12, ram_gb
        elif ram_gb > 20:  # Pro+ High RAM
            return 'Pro+ High-RAM', 10, ram_gb
        elif ram_gb > 12:  # Pro
            return 'Pro', 6, ram_gb
        else:  # Free
            return 'Free', 3, ram_gb
    except:
        return 'Unknown', 5, 0

tier, auto_workers, ram_gb = detect_colab_tier()

# Detect GPU
try:
    import subprocess
    gpu_info = subprocess.check_output(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader']).decode()
    gpu_name = gpu_info.strip()
except:
    gpu_name = 'No GPU'

print('🔥' + '='*48 + '🔥')
print(f'🚀 DETECTED: {tier.upper()}')
print(f'💾 System RAM: {ram_gb:.1f} GB')
print(f'🎮 GPU: {gpu_name}')
print(f'⚡ Auto-set: {auto_workers} workers (MAX POWER!)')
print('🔥' + '='*48 + '🔥')

style = {'description_width': '180px'}
layout = Layout(width='600px')
layout_short = Layout(width='180px')

dest_text = widgets.Text(description='📁 Folder đích:', placeholder='Link folder Google Drive của bạn', style=style, layout=layout)
source_text = widgets.Text(description='📂 Folder nguồn:', placeholder='Link folder cần copy', style=style, layout=layout)
from_page_text = widgets.Text(description='Từ trang:', value='0', style=style, layout=layout_short)
to_page_text = widgets.Text(description='Đến trang:', value='0', style=style, layout=layout_short)
max_download_size_text = widgets.Text(description='💾 Giới hạn (GB):', value='0', placeholder='0 = không giới hạn', style=style, layout=layout)
exclude_str_text = widgets.Text(description='🚫 Bỏ qua chứa:', value='', placeholder='.tmp, backup', style=style, layout=layout)
include_ext_text = widgets.Text(description='✅ Chỉ copy đuôi:', value='', placeholder='.pdf, .mp4', style=style, layout=layout)
exclude_ext_text = widgets.Text(description='🚫 Bỏ qua đuôi:', value='', placeholder='.tmp, .log', style=style, layout=layout)
parallel_text = widgets.Text(description='⚡ Số luồng:', value=str(auto_workers), style=style, layout=layout_short)
dry_run_checkbox = widgets.Checkbox(description='🔍 Dry-run (chỉ xem)', value=False, layout=Layout(width='300px'))
overwrite_checkbox = widgets.Checkbox(description='📝 Ghi đè nếu size khác', value=False, layout=Layout(width='300px'))

display(HTML('<b>📌 Thông tin chính:</b>'))
display(dest_text, source_text)
display(HTML('<b>📄 Phân trang:</b>'))
display(HBox([from_page_text, to_page_text]))
display(HTML('<b>⚙️ Cấu hình:</b>'))
display(max_download_size_text, parallel_text)
display(HTML('<b>🔍 Bộ lọc:</b>'))
display(exclude_str_text, include_ext_text, exclude_ext_text)
display(HTML('<b>🎛️ Tùy chọn:</b>'))
display(dry_run_checkbox, overwrite_checkbox)
print('-' * 50)
print('✅ Nhập xong thì chạy cell tiếp theo!')

In [ ]:
#@title 3️⃣ Run - GDrive Turbo Copy
import os, time, re, json, gc
from datetime import datetime
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import Lock, Event
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.colab import auth
from tqdm.notebook import tqdm

class GDriveTurboCopy:
    def __init__(self):
        self._total_size = 0
        self._limit_size = 0
        self._copied_count = 0
        self._skipped_count = 0
        self._error_count = 0
        self._total_files = 0
        self._start_time = 0
        self._lock = Lock()
        self._stop = Event()
        self.excluded_strings = []
        self.include_ext = []
        self.exclude_ext = []
        self.workers = 5
        self.dry_run = False
        self.overwrite = False
        self.checkpoint = '/content/copy_checkpoint.json'
        self.logfile = '/content/copy_log.txt'
        self.copied_files = set()
        self.service = None

    def log(self, msg, level='INFO'):
        ts = datetime.now().strftime('%H:%M:%S')
        print(f'[{ts}] {msg}')
        try:
            with open(self.logfile, 'a', encoding='utf-8') as f:
                f.write(f'[{datetime.now()}] [{level}] {msg}\n')
        except: pass

    def play_sound(self, success=True):
        try:
            from IPython.display import Audio, display
            import numpy as np
            sr = 22050
            t = np.linspace(0, 0.1, int(sr*0.1))
            if success:
                sound = np.sin(2*np.pi*800*t) * np.exp(-5*t)
            else:
                sound = np.sin(2*np.pi*300*t) * np.exp(-3*t)
            display(Audio(sound, rate=sr, autoplay=True))
        except: pass

    def format_time(self, sec):
        if sec < 60: return f'{int(sec)}s'
        if sec < 3600: return f'{int(sec//60)}m{int(sec%60)}s'
        return f'{int(sec//3600)}h{int((sec%3600)//60)}m'

    def get_eta(self):
        if self._copied_count == 0 or self._total_files == 0: return '...'
        elapsed = time.time() - self._start_time
        if elapsed <= 0: return '...'
        rate = self._copied_count / elapsed
        remain = self._total_files - self._copied_count - self._skipped_count
        return self.format_time(remain / rate) if rate > 0 and remain > 0 else '~'

    def load_checkpoint(self):
        try:
            if os.path.exists(self.checkpoint):
                with open(self.checkpoint, 'r') as f:
                    d = json.load(f)
                    self.copied_files = set(d.get('files', []))
                    self._total_size = d.get('size', 0)
                    self._copied_count = d.get('copied', 0)
                    self._skipped_count = d.get('skipped', 0)
                    self._total_files = d.get('total', 0)
                    self.log(f'✅ Resume: {len(self.copied_files)} files, {self._total_size/1024:.2f}GB')
                    return True
        except: pass
        return False

    def save_checkpoint(self):
        try:
            with self._lock:
                with open(self.checkpoint, 'w') as f:
                    json.dump({'files': list(self.copied_files), 'size': self._total_size, 'copied': self._copied_count, 'skipped': self._skipped_count, 'total': self._total_files}, f)
        except: pass

    def auth(self):
        self.log('🔐 Đang xác thực...')
        auth.authenticate_user()
        self.service = build('drive', 'v3')
        self.log('✅ Xác thực thành công')

    def sanitize(self, name):
        if not name: return 'unnamed'
        for c in ['/', '\\', ':', '*', '?', '"', '<', '>', '|']: name = name.replace(c, '_')
        name = ''.join(c for c in name if c.isprintable() or ord(c) > 127)
        if len(name) > 200:
            ext = os.path.splitext(name)[1]
            name = name[:200-len(ext)] + ext
        return name.strip() or 'unnamed'

    def should_include(self, name):
        if not name: return True
        ext = os.path.splitext(name)[1].lower()
        if self.include_ext: return ext in self.include_ext
        if self.exclude_ext: return ext not in self.exclude_ext
        return True

    def retry(self, func, retries=5, delay=2):
        for i in range(retries):
            try: return func()
            except HttpError as e:
                st = e.resp.status
                if st == 404: return None
                if st in [403, 429, 500, 502, 503, 504]:
                    wait = min(delay * (2 ** i), 60)
                    self.log(f'⚠️ API {st}, retry {i+1}/{retries} in {wait}s...', 'WARN')
                    time.sleep(wait)
                else: raise
            except Exception as e:
                if i < retries - 1: time.sleep(min(delay * (2 ** i), 30))
                else: raise
        return None

    def list_folder(self, fid):
        files = []
        token = None
        q = f"'{fid}' in parents and trashed=false"
        for s in self.excluded_strings: q += f" and not name contains '{s}'"
        while not self._stop.is_set():
            try:
                r = self.retry(lambda: self.service.files().list(q=q, orderBy='folder,name', pageToken=token, pageSize=1000, fields='files(id,name,mimeType,size),nextPageToken', supportsAllDrives=True, includeItemsFromAllDrives=True).execute())
                if not r: break
                for f in r.get('files', []):
                    m = f.get('mimeType', '')
                    if m == 'application/vnd.google-apps.folder': files.append(f)
                    elif self.should_include(f.get('name', '')):
                        if not m.startswith('application/vnd.google-apps.'): files.append(f)
                token = r.get('nextPageToken')
                if not token: break
            except Exception as e:
                self.log(f'❌ List error: {e}', 'ERROR')
                break
        return files

    def exists(self, dest_id, name, size=None):
        try:
            esc = name.replace("'", "\\'")
            q = f"'{dest_id}' in parents and name='{esc}' and trashed=false"
            r = self.retry(lambda: self.service.files().list(q=q, fields='files(id,size)', supportsAllDrives=True, pageSize=1).execute())
            if r and r.get('files'):
                f = r['files'][0]
                if self.overwrite and size is not None and int(f.get('size', 0)) != size:
                    try: self.service.files().delete(fileId=f['id'], supportsAllDrives=True).execute()
                    except: pass
                    return None
                return f['id']
        except: pass
        return None

    def copy_file(self, dest_id, src):
        if self._stop.is_set(): return 'stop'
        fid, name, size = src['id'], self.sanitize(src.get('name', 'unnamed')), int(src.get('size', 0))
        mb, key = size / (1024*1024), f'{dest_id}:{fid}'
        if key in self.copied_files:
            with self._lock: self._skipped_count += 1
            return 'skip'
        if self.dry_run:
            self.log(f'[DRY] {name} ({mb:.1f}MB)')
            with self._lock: self._total_size += mb
            return 'dry'
        if self.exists(dest_id, name, size):
            with self._lock: self._skipped_count += 1; self.copied_files.add(key)
            return 'exists'
        try:
            t0 = time.time()
            r = self.retry(lambda: self.service.files().copy(body={'parents': [dest_id], 'name': name}, fileId=fid, supportsAllDrives=True, fields='id').execute())
            if r:
                spd = mb / max(time.time() - t0, 0.1)
                with self._lock: self._total_size += mb; self._copied_count += 1; self.copied_files.add(key)
                eta = self.get_eta()
                self.log(f'✅ {name} ({mb:.1f}MB, {spd:.1f}MB/s) ETA:{eta}')
                if self._copied_count % 10 == 0: self.save_checkpoint(); gc.collect()
                if self._limit_size > 0 and self._total_size >= self._limit_size * 1024:
                    self._stop.set(); return 'limit'
                return 'ok'
        except Exception as e:
            with self._lock: self._error_count += 1
            self.log(f'❌ {name}: {str(e)[:80]}', 'ERROR')
        return 'error'

    def create_folder(self, dest_id, name):
        name = self.sanitize(name)
        eid = self.exists(dest_id, name)
        if eid: return eid
        if self.dry_run: return f'dry_{name}'
        try:
            r = self.retry(lambda: self.service.files().create(body={'name': name, 'mimeType': 'application/vnd.google-apps.folder', 'parents': [dest_id]}, fields='id', supportsAllDrives=True).execute())
            if r: self.log(f'📁 {name}'); return r['id']
        except Exception as e: self.log(f'❌ Folder {name}: {e}', 'ERROR')
        return None

    def copy_contents(self, dest_id, src_id, depth=0):
        if self._stop.is_set(): return
        items = self.list_folder(src_id)
        if not items: return
        folders = [f for f in items if f.get('mimeType') == 'application/vnd.google-apps.folder']
        files = [f for f in items if f.get('mimeType') != 'application/vnd.google-apps.folder']
        self.log(f'{"  "*depth}📊 {len(files)} files, {len(folders)} folders')
        self._total_files += len(files)
        if files and not self._stop.is_set():
            with tqdm(total=len(files), desc='Copying', leave=True) as pbar:
                with ThreadPoolExecutor(max_workers=self.workers) as ex:
                    futs = {ex.submit(self.copy_file, dest_id, f): f for f in files}
                    for fut in as_completed(futs):
                        pbar.update(1)
                        if fut.result() in ['limit', 'stop']:
                            self._stop.set()
                            break
        for folder in folders:
            if self._stop.is_set(): break
            fname = folder.get('name', 'unnamed')
            self.log(f'{"  "*depth}📁 {fname}')
            sub_id = self.create_folder(dest_id, fname)
            if sub_id and not sub_id.startswith('dry_'): self.copy_contents(sub_id, folder['id'], depth+1)
        gc.collect()

    def extract_id(self, url):
        if not url: return None
        for p in [r'/folders/([a-zA-Z0-9_-]{25,})', r'id=([a-zA-Z0-9_-]{25,})', r'^([a-zA-Z0-9_-]{25,})$']:
            m = re.search(p, url)
            if m: return m.group(1)
        m = re.search(r'([a-zA-Z0-9_-]{25,})', url)
        return m.group(1) if m else None

    def get_info(self, fid):
        try: return self.service.files().get(fileId=fid, fields='id,name,mimeType', supportsAllDrives=True).execute()
        except: return None

    def run(self, dest_url, src_url):
        self._start_time = time.time()
        self.log('='*50)
        self.log('🚀 GDRIVE TURBO COPY')
        self.log('='*50)
        self.auth()
        self.load_checkpoint()
        self.log(f'⚙️ Workers: {self.workers} | Limit: {self._limit_size}GB | Dry: {self.dry_run}')
        dest_id, src_id = self.extract_id(dest_url), self.extract_id(src_url)
        if not dest_id or not src_id: self.log('❌ URL không hợp lệ!', 'ERROR'); return
        src_info = self.get_info(src_id)
        if not src_info: self.log('❌ Không truy cập được folder nguồn!', 'ERROR'); return
        dest_info = self.get_info(dest_id)
        if not dest_info: self.log('❌ Không truy cập được folder đích!', 'ERROR'); return
        self.log(f'📂 Nguồn: {src_info.get("name")}')
        self.log(f'📂 Đích: {dest_info.get("name")}')
        self.log('-'*50)
        new_dest = self.create_folder(dest_id, src_info.get('name', 'Copy'))
        if not new_dest: self.log('❌ Không tạo được folder đích!', 'ERROR'); return
        self.copy_contents(new_dest, src_id)
        self.save_checkpoint()
        elapsed = time.time() - self._start_time
        gb, spd = self._total_size / 1024, self._total_size / max(elapsed, 1)
        self.log('='*50)
        self.log('📊 KẾT QUẢ')
        self.log(f'✅ Copied: {self._copied_count} | ⏭️ Skipped: {self._skipped_count} | ❌ Errors: {self._error_count}')
        self.log(f'💾 {gb:.2f}GB | ⏱️ {self.format_time(elapsed)} | 🚀 {spd:.1f}MB/s')
        self.log('='*50)
        self.play_sound(self._error_count == 0)
        if self._error_count == 0: self.log('🎉 HOÀN TẤT!')
        else: self.log(f'⚠️ Có {self._error_count} lỗi. Chạy lại để retry.')

# === MAIN ===
print("🔧 Khởi tạo...")
dl = GDriveTurboCopy()
dl._limit_size = float(max_download_size_text.value) if max_download_size_text.value else 0
dl.excluded_strings = [s.strip() for s in exclude_str_text.value.split(',') if s.strip()]
dl.include_ext = [s.strip().lower() if s.strip().startswith('.') else f'.{s.strip().lower()}' for s in include_ext_text.value.split(',') if s.strip()]
dl.exclude_ext = [s.strip().lower() if s.strip().startswith('.') else f'.{s.strip().lower()}' for s in exclude_ext_text.value.split(',') if s.strip()]
dl.workers = max(1, min(15, int(parallel_text.value))) if parallel_text.value else 5
dl.dry_run = dry_run_checkbox.value
dl.overwrite = overwrite_checkbox.value
if not dest_text.value.strip(): print("❌ Nhập link folder đích!")
elif not source_text.value.strip(): print("❌ Nhập link folder nguồn!")
else: dl.run(dest_text.value.strip(), source_text.value.strip())


In [ ]:
#@title 4 Tiện ích - Xóa checkpoint
import os
for f in ['/content/copy_checkpoint.json', '/content/copy_log.txt', '/content/copy_errors.txt']:
    if os.path.exists(f):
        os.remove(f)
        print(f' Đã xóa: {f}')
    else:
        print(f' Không có: {f}')
print('\n Có thể chạy lại từ đầu!')